In [4]:
import torchhd
import torch

def euclidean(x: torch.Tensor, y: torch.Tensor) -> torch.Tensor:
    """Compute pairwise Euclidean distance between hypervectors."""
    return torch.cdist(x.unsqueeze(0), y.unsqueeze(0)).squeeze(0)

def euclid(x, y):
    """Drop-in replacement for torchhd.cos, returns negative Euclidean distance."""
    return -torch.cdist(x, y)

a = torchhd.random(1, 10000)  # 10 hypervectors
b = torchhd.random(5, 10000)   # 5 hypervectors

# Cosine similarity (built-in)
sim = torchhd.cos(a, b)
print(sim.tolist())
# Euclidean distance (custom)
dist = euclidean(a, b)
print(dist.tolist())

sim_e = euclid(a, b)
print(sim_e.tolist())

[[-0.015399999916553497, 0.009399999864399433, 0.004399999976158142, 0.006800000090152025, -0.0031999999191612005]]
[[142.50613403320312, 140.75511169433594, 141.10989379882812, 140.939697265625, 141.64744567871094]]
[[0.007017241790890694, 0.007104537915438414, 0.007086675148457289, 0.00709523307159543, 0.007059781346470118]]


In [1]:
import random

import torch
from gitdb.fun import chunk_size

from src.encoding.configs_and_constants import ZINC_SMILES_HRR_7744_CONFIG
from src.encoding.graph_encoders import load_or_create_hypernet, HyperNet
from src.utils.utils import GLOBAL_MODEL_PATH

cfg = ZINC_SMILES_HRR_7744_CONFIG

e1 = load_or_create_hypernet(path=GLOBAL_MODEL_PATH, cfg=cfg, depth=3, use_edge_codebook=False)
e1.populate_codebooks()

Loading existing HyperNet from /Users/arvandkaveh/Projects/kit/graph_hdc/_models/hypernet_ZincSmilesHRR7744_HRR_dim7744_s42_depth3_ecb0.pt


In [1]:
import numpy as np
arr = np.load("/home/akaveh/Projects/kit/graph_hdc/_artefacts/cond_generation/qm9_nvp_qm9_h1600_f12_hid1024_s42_lr5e-4_wd0.0_an_SIMPLE_VOTER/0.305.npy")
print(len(arr))

4


In [2]:
e2 = load_or_create_hypernet(path=GLOBAL_MODEL_PATH, cfg=cfg, depth=3, use_edge_codebook=False)

Loading existing HyperNet from /Users/arvandkaveh/Projects/kit/graph_hdc/_models/hypernet_ZincSmilesHRR7744_HRR_dim7744_s42_depth3_ecb0.pt


In [3]:
from src.encoding.configs_and_constants import Features
import torch

node_cb1 = e1.node_encoder_map[Features.ATOM_TYPE][0].codebook
node_cb2 = e2.node_encoder_map[Features.ATOM_TYPE][0].codebook



assert torch.equal(e1.node_encoder_map[Features.ATOM_TYPE][0].codebook, e2.node_encoder_map[Features.ATOM_TYPE][0].codebook)

assert torch.equal(e1.nodes_codebook, e2.nodes_codebook)
assert torch.equal(e1.nodes_codebook, e2.node_encoder_map[Features.ATOM_TYPE][0].codebook)


In [6]:
from src.datasets.zinc_smiles_generation import ZincSmiles

for s in ['valid', 'test', 'train']:
    ds = ZincSmiles(split=s, enc_suffix="HRR7744")
    # HRRTensor([ 0.0589,  0.2546,  0.0979,  ..., -0.0511,  0.0170, -0.0878])
    node = ds.node_terms
    graph = ds.graph_terms

    def fmt_stats(t):
        return (
            f"- min: {t.min().item():+.4f}  max: {t.max().item():+.4f}  mean: {t.mean().item():+.4f} Median: {t.median().item():+.4f}\n"
        )


    print(
        f"---------{s}----------"
        f"ZincSmiles({s}, enc_suffix='HRR7744')\n"
        ".. rubric:: Encoded tensor statistics\n\n"
        "**Node terms**\n"
        f"{fmt_stats(node)}"
        "**Graph terms**\n"
        f"{fmt_stats(graph)}"
)

SyntaxError: Invalid star expression (623273207.py, line 8)

--------- valid ----------
ZincSmiles('valid', enc_suffix='HRR7744')
N = 24,445, node shape = (24445, 7744), graph shape = (24445, 7744)
.. rubric:: Encoded tensor statistics



RuntimeError: quantile() input tensor is too large

In [31]:
from collections import defaultdict
import random
import numpy as np

a = random.sample(range(100), k=20)
# assert len(a) == len(set(a))
#
# chunk_size = 20
# n_chunks = 5
#
# chunks = defaultdict(list)
# for i in a:
#     chunk_number = i // chunk_size
#     chunks[chunk_number].append(i)
#
# for l in chunks.values():
#     random.shuffle(l)
#
# final = []
# for l in range(n_chunks):
#     final.extend(chunks[l])

a = np.array(a, dtype=np.int32)
chunk_id = a // chunk_size
keys = chunk_id + np.random.random(a.size)
order = np.argsort(keys)
final = a[order]
